## Preparando o Ambiente

#### Importando as Bibliotecas

In [95]:
import torch  # Biblioteca para computação com tensores e deep learning (usada com BERT)

import nltk     # Biblioteca com stop-words para o Português

import requests  # Realiza requisições HTTP (ex: baixar arquivos ou acessar APIs)

import pandas as pd  # Manipulação de dados em tabelas (DataFrames)

import numpy as np  # Operações matemáticas com arrays e vetores

from sklearn.metrics.pairwise import cosine_similarity  # Mede a similaridade entre vetores (ex: consulta vs documentos)

from sklearn.feature_extraction.text import (
    CountVectorizer,    # Transforma texto em vetores de contagem de palavras
    TfidfVectorizer     # Transforma texto em vetores com pesos TF-IDF
)

from sklearn.decomposition import (
    TruncatedSVD,  # Redução de dimensionalidade usando decomposição SVD
    NMF            # Redução de dimensionalidade com apenas valores positivos
)

from transformers import (
    BertModel,     # Modelo pré-treinado BERT para gerar embeddings semânticos
    BertTokenizer  # Tokenizador que converte texto em formato que o BERT entende
)


#### Baixando  os dados

In [3]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

#### Criando um dataframe com os dados baixados

In [5]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


## Busca Textual

### Conceitos:

* **Recuperação de Informação:** O processo de obtenção de informações relevantes de grandes conjuntos de dados com base em consultas de usuários.
* **Espaços Vetoriais:** Uma representação matemática em que o texto é convertido em vetores (pontos no espaço), permitindo a comparação quantitativa.
* **Bag of Words:** Um modelo simples de representação de texto que trata cada documento como uma coleção de palavras, desconsiderando a gramática e a ordem das palavras, mas mantendo a multiplicidade.
* **TF-IDF (Frequência de Termos - Frequência Inversa do Documento):** Uma medida estatística usada para avaliar a importância de uma palavra para um documento em uma coleção ou corpus. Ela aumenta com o número de vezes que uma palavra aparece no documento, mas é compensada pela frequência da palavra no corpus.

### Busca por Palavras-Chaves

In [6]:
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


### Vetorização por contagem de palavras (Bag-of-words)

O modelo `Bag-of-words` é uma representação simplificada utilizada no processamento de linguagem natural e na recuperação de informações. Neste modelo, o texto é representado como um multiconjunto de suas palavras, desconsiderando a estrutura gramatical e até mesmo a ordenação delas, mas mantendo sua multiplicidade.

Documento de exemplo:

In [76]:
docs_example = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

Cria um objeto `CountVectorizer`, que transforma um texto em uma matriz de contagem de palavras, removendo palavras comuns da língua inglesa.

In [77]:
cv = CountVectorizer(stop_words='english')
cv

CountVectorizer(stop_words='english')

Transforma o texto dos documentos em uma matriz de contagem de palavras.

In [78]:
X = cv.fit_transform(docs_example)
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 20 stored elements and shape (5, 19)>

Extrai a lista de todas as palavras únicas encontradas nos documentos.

In [79]:
names = cv.get_feature_names_out()
names

array(['15th', '2024', 'cloud', 'course', 'date', 'github', 'google',
       'homeworks', 'jan', 'listed', 'participation', 'prerequisites',
       'python', 'registration', 'required', 'setup', 'start', 'starts',
       'submit'], dtype=object)

Cria um dataframe com a contagem da palavra em cada documento.

In [80]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


Criando vetor e dataframe de consulta

In [ ]:
query = ['python course start date']
X_query = cv.transform(query)
df_query = pd.DataFrame(X_query.toarray(), columns=names).T
df_query

Calculando a similaridade usando o produto escalar (soma das multiplicações palavra a palavra), onde cada score indica o número de palavras em comum (e sua frequência) entre a consulta e o documento.

In [91]:
scores = df_docs.values.T @ df_query.values
scores

array([[1],
       [0],
       [2],
       [0],
       [2]])

Retornando o resultado, onde quanto maior o score, mais relevante o documento.

In [92]:
df_results = pd.DataFrame({
    'documento': docs_example,
    'score': scores.flatten()
}).sort_values(by='score', ascending=False)
df_results

,documento,score
2,Submit homeworks after start date,2
4,Setup Google Cloud and Python before course,2
0,Course starts on 15th Jan 2024,1
1,Prerequisites listed on GitHub,0
3,Registration not required for participation,0


#### Exercício: Buscador de receitas simples

Você recebeu uma base com descrições curtas de receitas culinárias. Seu objetivo é criar uma função de busca que, dado um termo ou frase de consulta (query), retorne as 3 receitas mais relevantes com base na similaridade textual por saco de palavras (bag of words).

In [103]:
receitas = [
    "Bolo de chocolate com cobertura de brigadeiro",
    "Salada de alface, tomate e cenoura com azeite",
    "Lasanha de carne moída com molho branco e queijo",
    "Sopa de legumes com batata, cenoura e mandioquinha",
    "Frango assado com batatas e alecrim",
    "Torta de maçã com canela e massa folhada",
    "Arroz de forno com presunto, ervilha e queijo",
    "Macarrão ao alho e óleo com pimenta",
    "Panqueca recheada com frango e catupiry",
    "Brigadeiro tradicional de festa",
    "Feijoada completa com arroz, couve e farofa",
    "Escondidinho de carne seca com purê de mandioca",
    "Moqueca de peixe com leite de coco e coentro",
    "Quiche de espinafre com ricota",
    "Omelete de queijo, tomate e orégano",
    "Cuscuz nordestino com ovo e manteiga de garrafa",
    "Risoto de cogumelos com parmesão",
    "Carne de panela com batata e cenoura",
    "Pão de queijo mineiro",
    "Brownie de chocolate com nozes"
]


In [96]:
nltk.download('stopwords')
stopwords_pt = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/elladarte/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [104]:
def buscar_receitas(query: str) -> list[str]:
    cv = CountVectorizer(stop_words=stopwords_pt)
    X = cv.fit_transform(receitas)
    palavras_receitas = cv.get_feature_names_out()
    df_receitas = pd.DataFrame(X.toarray(), columns=palavras_receitas).T
    X_query = cv.transform([query])
    df_query = pd.DataFrame(X_query.toarray(), columns=palavras_receitas).T
    scores = df_receitas.values.T @ df_query.values
    df_results = pd.DataFrame({
        'documento': receitas,
        'score': scores.flatten()
    }).sort_values(by='score', ascending=False)

    return df_results['documento']
    


In [105]:
result = buscar_receitas('chocolate')
result.head(3)

0        Bolo de chocolate com cobertura de brigadeiro
19                      Brownie de chocolate com nozes
2     Lasanha de carne moída com molho branco e queijo
Name: documento, dtype: object

### TF-IDF: Term Frequency – Inverse Document Frequency

### Resumo Comparativo

| Tipo de Busca             | Vetor usado                     | Reconhece sinônimos? | Considera contexto? | Peso por importância? | Complexidade | Exemplo de uso                       |
| ------------------------- | ------------------------------- | -------------------- | ------------------- | --------------------- | ------------ | ------------------------------------ |
| **Palavra-chave simples** | Nenhum (string in documento)    | ❌ Não                | ❌ Não               | ❌ Não                 | Muito baixa  | `"chocolate" in doc.lower()`         |
| **Bag of Words**          | Frequência de palavras          | ❌ Não                | ❌ Não               | ❌ Não                 | Baixa        | `CountVectorizer`                    |
| **TF-IDF**                | Frequência + importância global | ❌ Não                | ⚠️ Parcialmente     | ✅ Sim                 | Média        | `TfidfVectorizer`                    |



## Busca Vetorial

#### Similaridade de cosseno

### Técnicas de redução de dimensionalidade

#### SVD: Singular Value Decomposition

#### NMF: Non-Negative Matrix Factorization

### BERT: Bidirectional Encoder Representations from Transformers

#### Embeddings

## Busca Textual + Busca Vetorial

### TF-IDF + BERT

## Técnicas para buscas eficientes

### Índice Invertido

### LSH: Locality-Sensitive Hashing

### Random projection